In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

dlist = []
y = []
count = 0
with open('ppdata.csv', 'r', encoding='utf-8') as r:
    count = 0
    for l in r:
        if count > 0:
            #print(type(l))
            l = l.split(',')
            #print(l)
            val = l[0]
            wds = l[1]
            wrds = wds.split(' ')
            #f = [val, wrds]
            dlist.append(wrds)
            y.append([np.asarray(int(val), dtype=np.int32)])
            count += 1
        else:
            count += 1
        #if count > 10:
            #break
            
y = np.array(y)
y = y.ravel()

max_words = 12000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(dlist)
sequences = tokenizer.texts_to_sequences(dlist)
x = pad_sequences(sequences, maxlen=max_len)
x = x.astype(np.float32)

Using TensorFlow backend.


In [2]:
shuffle_ind = np.random.permutation(len(x))  # shuffle data for training
x = np.array(x[shuffle_ind])

x /= np.max(x)  # linearly scaling the inputs to a range between 0 and 1

train_fraction = 0.75
train_idx = int(len(x)*train_fraction)

x_train = x[:train_idx]
y_train = y[:train_idx]

x_val = x[train_idx:]
y_val = y[train_idx:]

In [3]:
def mse(y, pred):
    """Mean squared error function comparing real target values with predictions.
    Args:
      y: List containing real target values 
      pred: List containing predicted target values (must have the same shape as y)
    Returns:
      Mean squared error between the two arrays
    """
    return np.mean(np.square(y - pred))

In [5]:
clf = LogisticRegression(random_state=0, solver='sag').fit(x_train, y_train)
scores = cross_val_score(clf, x_val, y_val, cv=5)
pred = clf.predict(x_train)
val_loss = mse(pred, y_train)
print('5 fold cross validation score:', sum(scores)/len(scores))
print('validation loss:', val_loss)
print('validation accuracy', clf.score(x, y))

5 fold cross validation score: 0.49504000000000004
validation loss: 0.4747733333333333
validation accuracy 0.52052
